# Comparison Notebook

The last few cells of code including the big pandas dataframe compare different players

In [1]:
%matplotlib inline
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import pandas as pd

from game import Game
from player import Player

In [2]:
from your_name import YourName

In [3]:
### Generate cards from 9 to 14 (ace) for all colors/symbols (0, 1, 2, 3)
def getDeck():
    return [(number, color) for color in range(4) for number in range(9, 15)]
    
print(getDeck())

[(9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 2), (9, 3), (10, 3), (11, 3), (12, 3), (13, 3), (14, 3)]


In [4]:
### Shuffle the cards randomly. Each player gets 9 cards
### (so one player cannot be certain which cards the other player has)

def getShuffled(deck):
    D = set(deck)
    A = set(random.sample(deck, 8))
    B = set(random.sample(list(D - A), 8))
    C = D - A - B
    if len(A.intersection(B)) > 0: print("Shuffle error 1")
    if len(A.intersection(B)) > 0: print("Shuffle error 2")
    if len(A.intersection(C)) > 0: print("Shuffle error 3") 
    DS = A | B | C
    if not DS == D: print("Shuffle error 4")  
    return list(A), list(B), list(C)

p1, p2, notUsed, = getShuffled(getDeck())
print(p1)
print(p2)


[(9, 0), (14, 1), (12, 0), (9, 2), (13, 0), (11, 2), (10, 3), (10, 2)]
[(11, 1), (9, 3), (10, 1), (12, 3), (14, 0), (14, 3), (13, 2), (14, 2)]


In [5]:
# Some examplary random player

class RandomPlayer(Player):
    
    ### player's random strategy
    def putCard(self, declared_card):
        
        ### check if must draw
        if len(self.cards) == 1 and declared_card is not None and self.cards[0][0] < declared_card[0]:
            return "draw"
        
        ### player randomly decides which card put on the table
        card = random.choice(self.cards)
        declaration = card
        
        ### player randomly decides whether to cheat or not
        cheat = np.random.choice([True, False])
       
        ### if (s)he decides to cheat, (s)he randomly declares the card.
        if cheat:
            declaration = random.choice(self.cards)             
            
        ### Yet, declared card should be no worse than a card on the top of the pile . 
        if declared_card is not None and declaration[0] < declared_card[0]:
            declaration = (min(declared_card[0]+1,14), declaration[1])

        ### return the decision (true card) and declaration (player's declaration)
        return card, declaration
    
    ### randomly decides whether to check or not
    def checkCard(self, opponent_declaration):
        return np.random.choice([True, False])
    

Analyze few moves...

In [6]:
### Perform a full game 100 times
STAT_NAMES = ["Wins", "Draws", "Moves", "Cards", "Pile Size", "Checks", "Draw Decisions", "Cheats", "Errors", "Total errors"]
ANALYZE_STATS = [0, 1, 2, 3, 5, 6, 7, 8]

def printResults(results):
    print("Wins:")
    print(results[0])
    print("Draws:")
    print(results[1])
    print("Moves:")
    print(stats_moves[2])
    print("Cards:")
    print(results[3])
    print("Pile size:")
    print(results[4])
    print("Checks:")
    print(results[5])
    print("Draw decisions:")
    print(results[6])
    print("Cheats:")
    print(results[7])
    print("Errors:")
    print(results[8])
    print("Total errors:")
    print(results[9])

def comparePlayers(player1_class, player2_class):
    stats_wins = [0, 0]
    stats_draws = [0, 0]
    stats_moves = [0, 0]
    stats_cards = [0, 0]
    stats_pile_size = 0
    stats_checks = [0, 0]
    stats_draw_decisions = [0, 0]
    stats_cheats = [0, 0]
    stats_errors = [0, 0]
    
    repeats = 10000 #For most accurate tests
    errors = 0
    draws = 0

    for t in range(repeats):
        player1 = player1_class("")
        player2 = player2_class("")
        game = Game([player1, player2], log = False)
    
        error = False
        draw = False
        
        while True:
            valid, player = game.takeTurn(log = False)
            if game.moves[0] > 100 or game.moves[1] > 100:
                draws += 1
                stats_draws[0] += 1
                stats_draws[1] += 1
                if (game.player_cards[0] < game.player_cards[1]):
                    stats_wins[0] += 1
                if (game.player_cards[0] > game.player_cards[1]):
                    stats_wins[1] += 1
                    
                    
                draw=True
                #print("DRAW")
                break
            if not valid:
                error = True
                stats_errors[player] += 1
                errors += 1
                break
            if game.isFinished(log = False):
                stats_wins[player] += 1
                break
            
        stats_pile_size += len(game.pile)
        if error: continue
        #if draw: continue
       
        for j in range(2):
            stats_moves[j] += game.moves[j]
            stats_cheats[j] += game.cheats[j]
            stats_checks[j] += game.checks[j]
            stats_draw_decisions[j] += game.draw_decisions[j]
            stats_cards[j] += len(game.player_cards[j])

    div = repeats - errors
    if div > 0:
            
        stats_pile_size /= div          
        for j in range(2):
            stats_moves[j] /= div
            stats_cheats[j] /= div
            stats_checks[j] /= div
            stats_draw_decisions[j] /= div
            stats_cards[j] /= div
            
    return [stats_wins, stats_draws, stats_moves, stats_cards, stats_pile_size, stats_checks, 
            stats_draw_decisions, stats_cheats, stats_errors, errors, draws]  


# COMPARE

In [7]:
strategy = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ]

In [8]:
#%pdb on
full_results = [[None for i in range(len(strategy))] for i in range(len(strategy))]

for A in range(len(strategy)):
    print("==== " + str(A), strategy[A][0])
    for B in range(A+1,len(strategy)):
        print(B, strategy[B][0])
        results = comparePlayers(strategy[A][2], strategy[B][2])
        full_results[A][B] = results
        


==== 0 RandomPlayer
1 YourName
==== 1 YourName


In [9]:
full_results

[[None,
  [[10000, 0],
   [0, 0],
   [8.0, 7.4929],
   [0.0, 15.0],
   1.0,
   [0.0, 0.0],
   [0.0, 7.4929],
   [2.6417, 0.0],
   [0, 0],
   0,
   0]],
 [None, None]]

Simple stats

In [10]:
def printMatrix(full_results, stat):
    print(STAT_NAMES[stat])
    S = " "
    for s in strategy: S += (str(s[1]) + " " )
    print(S)
    for A in range(len(strategy)):
        print(A)
        S = str(strategy[A][1]) + " "
        for B in range(len(strategy)):
            if A == B: S += "- "
            elif A < B:
                S += str(full_results[A][B][stat][0]) + " "
            else:
                S += str(full_results[B][A][stat][1]) + " "
        print(S)
    
for a in ANALYZE_STATS:
    printMatrix(full_results, a)



Wins
 RANDOM YourName 
0
RANDOM - 10000 
1
YourName 0 - 
Draws
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 
Moves
 RANDOM YourName 
0
RANDOM - 8.0 
1
YourName 7.4929 - 
Cards
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 15.0 - 
Checks
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 0.0 - 
Draw Decisions
 RANDOM YourName 
0
RANDOM - 0.0 
1
YourName 7.4929 - 
Cheats
 RANDOM YourName 
0
RANDOM - 2.6417 
1
YourName 0.0 - 
Errors
 RANDOM YourName 
0
RANDOM - 0 
1
YourName 0 - 


### __________________________________________My Code ______________________________________________

In [11]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["YourName", "YourName", YourName],
           ["RandomPlayer2", "RANDOM", RandomPlayer]]

def compare_all(agent_strategies, verbose = 1):
    wins = {agent_strategies[i][0]: {agent_strategies[i][0]: None} for i in range(len(agent_strategies))}
    for i in range(len(agent_strategies)):
        for ii in range(i+1, len(agent_strategies)):
            player1 = agent_strategies[i]
            player2 = agent_strategies[ii]
            strategy = [player1, player2]
            for A in range(len(strategy)):
                for B in range(A+1,len(strategy)):
                    results = comparePlayers(strategy[A][2], strategy[B][2])
                    full_results[A][B] = results
                    wins[strategy[A][0]][strategy[B][0]] = full_results[0][1][0][1]
                    wins[strategy[B][0]][strategy[A][0]] = full_results[0][1][0][0]
                    if verbose:
                        if full_results[0][1][8][0] > 0 or full_results[0][1][8][1] > 0 or full_results[0][1][9] > 0:
                            print(f"Errors occured in game: {strategy[A][0]} vs {strategy[B][0]}")
    df = pd.DataFrame(wins)
    df['|| Total Wins ||'] = df.fillna(0).sum(axis=1)
    df = df.sort_values(by='|| Total Wins ||', ascending=False)

    dominated_players = []
    temp_df = df.drop(columns=["|| Total Wins ||"])
    for i, row in temp_df.iterrows():
        num_dominated = 0
        for column in temp_df.columns:
            if i != column and temp_df[column][i] > temp_df[i][column]:
                num_dominated += 1
        dominated_players.append(num_dominated)
    df['|| Dominated Players ||'] = dominated_players
    return df

compare_all(agent_strategies)

,RandomPlayer,YourName,RandomPlayer2,|| Total Wins ||,|| Dominated Players ||
RandomPlayer,NaN,10000.0,5060.0,15060.0,2
RandomPlayer2,4940.0,10000.0,NaN,14940.0,1
YourName,0.0,NaN,0.0,0.0,0


## Player Comaprison

Below I have tested the players against each other, and different simple heuristics as well as my solutions. The dataframe orders players according to the total number of wins against remaining players ( the "|| Total Wins ||" column), top players are the best. However, you can also check how many players the agent was better than - "|| Dominated Players ||". The numbers in remaining columns show how many times your agent has won against someone, so if you look at the row from dataframe above as an example the agent called "RandomPlayer" and compare it with column "RandomPlayer2" you can see that RandomPlayer won against the RandomPlayer2 4968 games out of 10000, you can also see that RandomPlayer2 won 5032 games against RandomPlayer, etc.

Below the dataframe I also print for some most important players their row values to see which agents they won against and which they won against, etc.

In [12]:
from Test_Players import *
from B import *
from adamK import *
from Adam_player import *
from ensamble import *
from Michal_Janek_ver_1 import *
from Michal_Janek_ver_2 import *
from Vadym_Player import *
from Krzysztof_Player import *
from Samuel_Player import *

In [13]:
agent_strategies = [["RandomPlayer", "RANDOM", RandomPlayer],
            ["Saint_Sceptic_25_Naive", "SAINT", Saint_Sceptic_25_Naive],
            ["Saint_Sceptic_50_Naive", "SAINT", Saint_Sceptic_50_Naive],
            ["Saint_Sceptic_75_Naive", "SAINT", Saint_Sceptic_75_Naive],
            ["Saint_Sceptic_Adaptable_Naive", "SAINT", Saint_Sceptic_Adaptable_Naive],
            ["Mat_Player_ver_4", "ENSAMBLE", ensamble_player],
            ["YourName - not implemented Player", "YourName", YourName],
           ["MyRandom_Default", "CUSTOM_RANDOM", MyRandom_Default],
           ["MyRandom_All_Cards", "CUSTOM_RANDOM", MyRandom_All_Cards],
           ["MyRandom_Real_Cards", "CUSTOM_RANDOM", MyRandom_Real_Cards],
           ["Random_Never_Accuse", "CUSTOM_RANDOM", Random_Never_Accuse],
           ["Random_Always_Accuse", "CUSTOM_RANDOM", Random_Always_Accuse],
           ["Mimic", "CUSTOM_RANDOM", Mimic],
           ["Saint_Believer", "SAINT", Saint_Believer],
           ["Saint_Once_in_a_While", "SAINT", Saint_Once_in_a_While],
           ["Saint_Two_Face", "SAINT", Saint_Two_Face],
           ["Saint_Accountant", "SAINT", Saint_Accountant],
           ["Saint_Nervous", "SAINT", Saint_Nervous],
           ["Mat_Player_ver_1", "SAINT", Saint_Nervous_ver_2],
           ["Mat_Player_ver_2", "SAINT", Saint_Nervous_ver_3],
           ["Mat_Player_ver_3", "SAINT", Saint_Nervous_Memory],
           ["Adam_M_Player", "CUSTOM", Adam_player],
           ["AdamK_ver_1", "CUSTOM", AdamK],
           ["AdamK2_ver_2", "CUSTOM", AdamK2],
           ["Saint_Collector", "SAINT", Saint_Collector],
           ["Dabbler_Believer", "DABBLER", Dabbler_Believer],
           ["Dabbler_Once_in_a_While", "DABBLER", Dabbler_Once_in_a_While],
           ["Dabbler_Two_Face", "DABBLER", Dabbler_Two_Face],
           ["Dabbler_Accountant", "DABBLER", Dabbler_Accountant],
           ["Dabbler_Nervous", "DABBLER", Dabbler_Nervous],
           ["Dabbler_Collector", "DABBLER", Dabbler_Collector],
           ["Coin_Flipper_Believer", "COIN_FLIPPER", Coin_Flipper_Believer],
           ["Coin_Flipper_Once_in_a_While", "COIN_FLIPPER", Coin_Flipper_Once_in_a_While],
           ["Coin_Flipper_Two_Face", "COIN_FLIPPER", Coin_Flipper_Two_Face],
           ["Coin_Flipper_Accountant", "COIN_FLIPPER", Coin_Flipper_Accountant],
           ["Coin_Flipper_Nervous", "COIN_FLIPPER", Coin_Flipper_Nervous],
           ["Coin_Flipper_Collector", "COIN_FLIPPER", Coin_Flipper_Collector],
           ["Pathological_Believer", "PATHOLOGICAL", Pathological_Believer],
           ["Pathological_Once_in_a_While", "PATHOLOGICAL", Pathological_Once_in_a_While],
           ["Pathological_Two_Face", "PATHOLOGICAL", Pathological_Two_Face],
           ["Pathological_Accountant", "PATHOLOGICAL", Pathological_Accountant],
           ["Pathological_Nervous", "PUSHER", Pathological_Nervous],
           ["Pathological_Collector", "PUSHER", Pathological_Collector],
           ["Pusher_Believer", "PUSHER", Pusher_Believer],
           ["Pusher_Once_in_a_While", "PUSHER", Pusher_Once_in_a_While],
           ["Pusher_Two_Face", "PUSHER", Pusher_Two_Face],
           ["Pusher_Accountant", "PUSHER", Pusher_Accountant],
           ["Pusher_Nervous", "PUSHER", Pusher_Nervous],
           ["Pusher_Collector", "PUSHER", Pusher_Collector],
           ["B_1", "B_PLAYER", Blayer],
           ["B_2", "B_PLAYER", Blayer2],
           ["B_3", "B_PLAYER", Blayer3],
           ["Bartek_ver_1", "B_PLAYER", Saint_Nervous_ver_B],
           ["Bartek_ver_2", "B_PLAYER", Saint_Nervous_ver_B2],
           ["Bartek_ver_3", "B_PLAYER", Saint_Nervous_ver_B3],
           ["Bartek_ver_4", "B_PLAYER", Saint_Nervous_ver_B4],
           ["Janek_Michal_ver_1", "MICHAL_JANEK", KaminskiIndrzejczak_ver_1],
           ["Janek_Michal_ver_2", "MICHAL_JANEK", KaminskiIndrzejczak_ver_2],
           ["Vadym_Player", "VADYM", Vadym_Player],
           ["Krzysztof_Player", "KRZYSZTOF", Krzysztof_Player],
           ["Samuel_Player", "SAMUEL", Amalgamat]
           ]

results = compare_all(agent_strategies)
results

,RandomPlayer,Saint_Sceptic_25_Naive,Saint_Sceptic_50_Naive,Saint_Sceptic_75_Naive,Saint_Sceptic_Adaptable_Naive,Mat_Player_ver_4,YourName - not implemented Player,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,...,Bartek_ver_2,Bartek_ver_3,Bartek_ver_4,Janek_Michal_ver_1,Janek_Michal_ver_2,Vadym_Player,Krzysztof_Player,Samuel_Player,|| Total Wins ||,|| Dominated Players ||
Mat_Player_ver_4,9998.0,10000.0,10000.0,10000.0,8256.0,NaN,10000.0,10000.0,10000.0,9999.0,...,4939.0,5353.0,5469.0,10000.0,3948.0,9200.0,10000.0,4051.0,510884.0,57
Bartek_ver_2,10000.0,10000.0,10000.0,10000.0,8195.0,5061.0,10000.0,10000.0,10000.0,9999.0,...,NaN,5441.0,5421.0,10000.0,4000.0,9192.0,10000.0,4075.0,510669.0,55
Bartek_ver_1,9999.0,10000.0,10000.0,10000.0,8201.0,4925.0,10000.0,10000.0,10000.0,9998.0,...,5047.0,5342.0,5380.0,10000.0,4003.0,9197.0,10000.0,3976.0,510438.0,56
Mat_Player_ver_1,9999.0,10000.0,10000.0,10000.0,8273.0,4964.0,10000.0,9999.0,10000.0,9999.0,...,5055.0,4543.0,4538.0,10000.0,3965.0,9109.0,10000.0,4045.0,505739.0,52
Mat_Player_ver_2,9999.0,10000.0,10000.0,10000.0,8207.0,4932.0,10000.0,9998.0,10000.0,10000.0,...,5008.0,4551.0,4410.0,10000.0,3974.0,9116.0,10000.0,4035.0,505540.0,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Pathological_Collector,244.0,12.0,68.0,1735.0,716.0,61.0,10000.0,317.0,341.0,249.0,...,57.0,365.0,364.0,137.0,1356.0,611.0,1.0,1297.0,106733.0,9
Random_Never_Accuse,242.0,14.0,141.0,2151.0,691.0,90.0,10000.0,277.0,237.0,222.0,...,95.0,267.0,292.0,129.0,1293.0,557.0,0.0,1469.0,106258.0,10
Pathological_Believer,185.0,8.0,161.0,2071.0,650.0,103.0,10000.0,245.0,241.0,232.0,...,98.0,290.0,264.0,126.0,1284.0,565.0,1.0,1343.0,105715.0,11
Mimic,145.0,0.0,0.0,0.0,1787.0,0.0,10000.0,80.0,183.0,96.0,...,0.0,77.0,76.0,0.0,53.0,9.0,0.0,0.0,65485.0,1


In [34]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
results

,RandomPlayer,Saint_Sceptic_25_Naive,Saint_Sceptic_50_Naive,Saint_Sceptic_75_Naive,Saint_Sceptic_Adaptable_Naive,Mat_Player_ver_4,YourName - not implemented Player,MyRandom_Default,MyRandom_All_Cards,MyRandom_Real_Cards,Random_Never_Accuse,Random_Always_Accuse,Mimic,Saint_Believer,Saint_Once_in_a_While,Saint_Two_Face,Saint_Accountant,Saint_Nervous,Mat_Player_ver_1,Mat_Player_ver_2,Mat_Player_ver_3,Adam_M_Player,AdamK_ver_1,AdamK2_ver_2,Saint_Collector,Dabbler_Believer,Dabbler_Once_in_a_While,Dabbler_Two_Face,Dabbler_Accountant,Dabbler_Nervous,Dabbler_Collector,Coin_Flipper_Believer,Coin_Flipper_Once_in_a_While,Coin_Flipper_Two_Face,Coin_Flipper_Accountant,Coin_Flipper_Nervous,Coin_Flipper_Collector,Pathological_Believer,Pathological_Once_in_a_While,Pathological_Two_Face,Pathological_Accountant,Pathological_Nervous,Pathological_Collector,Pusher_Believer,Pusher_Once_in_a_While,Pusher_Two_Face,Pusher_Accountant,Pusher_Nervous,Pusher_Collector,B_1,B_2,B_3,Bartek_ver_1,Bartek_ver_2,Bartek_ver_3,Bartek_ver_4,Janek_Michal_ver_1,Janek_Michal_ver_2,Vadym_Player,Krzysztof_Player,Samuel_Player,|| Total Wins ||,|| Dominated Players ||
Mat_Player_ver_4,9998.0,10000.0,10000.0,10000.0,8256.0,NaN,10000.0,10000.0,10000.0,9999.0,9910.0,10000.0,10000.0,6010.0,7775.0,9963.0,9442.0,5011.0,5036.0,5068.0,7418.0,9985.0,6544.0,6799.0,6054.0,8620.0,9468.0,9995.0,9870.0,8433.0,8735.0,9530.0,9805.0,9999.0,9973.0,9490.0,9547.0,9897.0,9974.0,10000.0,9996.0,9902.0,9939.0,9358.0,9743.0,9998.0,9949.0,9383.0,9547.0,6578.0,6428.0,5424.0,5075.0,4939.0,5353.0,5469.0,10000.0,3948.0,9200.0,10000.0,4051.0,510884.0,57
Bartek_ver_2,10000.0,10000.0,10000.0,10000.0,8195.0,5061.0,10000.0,10000.0,10000.0,9999.0,9905.0,10000.0,10000.0,6008.0,7726.0,9963.0,9452.0,5068.0,4945.0,4992.0,7418.0,9988.0,6484.0,6642.0,6086.0,8724.0,9464.0,9997.0,9866.0,8440.0,8763.0,9480.0,9798.0,9999.0,9981.0,9477.0,9541.0,9902.0,9948.0,9999.0,9999.0,9921.0,9943.0,9357.0,9725.0,9997.0,9950.0,9372.0,9562.0,6519.0,6500.0,5431.0,4953.0,NaN,5441.0,5421.0,10000.0,4000.0,9192.0,10000.0,4075.0,510669.0,55
Bartek_ver_1,9999.0,10000.0,10000.0,10000.0,8201.0,4925.0,10000.0,10000.0,10000.0,9998.0,9908.0,10000.0,10000.0,5927.0,7706.0,9953.0,9464.0,5053.0,5118.0,4912.0,7489.0,9981.0,6452.0,6712.0,5985.0,8693.0,9463.0,9997.0,9871.0,8384.0,8734.0,9481.0,9805.0,9999.0,9982.0,9444.0,9582.0,9902.0,9965.0,9999.0,9998.0,9925.0,9924.0,9402.0,9749.0,9998.0,9954.0,9391.0,9562.0,6477.0,6539.0,5490.0,NaN,5047.0,5342.0,5380.0,10000.0,4003.0,9197.0,10000.0,3976.0,510438.0,56
Mat_Player_ver_1,9999.0,10000.0,10000.0,10000.0,8273.0,4964.0,10000.0,9999.0,10000.0,9999.0,9913.0,10000.0,9650.0,5859.0,7729.0,9962.0,9406.0,4976.0,NaN,5057.0,7550.0,9989.0,6445.0,6343.0,5966.0,8517.0,9371.0,9996.0,9818.0,8198.0,8557.0,9389.0,9769.0,9998.0,9956.0,9403.0,9485.0,9887.0,9953.0,10000.0,9999.0,9907.0,9898.0,9380.0,9688.0,10000.0,9932.0,9315.0,9529.0,6473.0,6541.0,4564.0,4882.0,5055.0,4543.0,4538.0,10000.0,3965.0,9109.0,10000.0,4045.0,505739.0,52
Mat_Player_ver_2,9999.0,10000.0,10000.0,10000.0,8207.0,4932.0,10000.0,9998.0,10000.0,10000.0,9902.0,10000.0,9630.0,5976.0,7652.0,9957.0,9415.0,4952.0,4943.0,NaN,7451.0,9989.0,6449.0,6294.0,5966.0,8435.0,9388.0,9994.0,9847.0,8220.0,8502.0,9432.0,9755.0,9998.0,9973.0,9427.0,9475.0,9908.0,9959.0,10000.0,9997.0,9918.0,9910.0,9352.0,9706.0,9998.0,9935.0,9323.0,9482.0,6475.0,6607.0,4630.0,5088.0,5008.0,4551.0,4410.0,10000.0,3974.0,9116.0,10000.0,4035.0,505540.0,52
Saint_Nervous,10000.0,10000.0,10000.0,10000.0,8117.0,4989.0,10000.0,9999.0,10000.0,9999.0,9812.0,10000.0,9629.0,5792.0,7684.0,9971.0,9406.0,NaN,5024.0,5048.0,7506.0,9987.0,6231.0,6308.0,5877.0,6900.0,8672.0,9995.0,9593.0,6545.0,7071.0,8798.0,9486.0,9998.0,9928.0,8785.0,8912.0,9802.0,9907.0,9999.0,9994.0,9816.0,9835.0,9226.0,9615.0,9996.0,9910.0,9183.0,9422.0,6510.0,6434.0,3781.0,4947.0,4932.0,3621.0,3610.0,10000.0,3909.0,9124.0,10000.0,4031.0,493666.0,52
Samuel_Player,9785.0,9588.0,9567.0,9609.0,7967.0,5949.0,10000.0,9963.0,9988.0,9725.0,8531.0

### Player-Wise Comparisons

In [14]:
for col, val in results.loc["Janek_Michal_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9818.0
Saint_Sceptic_25_Naive: 5601.0
Saint_Sceptic_50_Naive: 5594.0
Saint_Sceptic_75_Naive: 5727.0
Saint_Sceptic_Adaptable_Naive: 0.0
Mat_Player_ver_4: 0.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9983.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9711.0
Random_Never_Accuse: 9871.0
Random_Always_Accuse: 9997.0
Mimic: 10000.0
Saint_Believer: 496.0
Saint_Once_in_a_While: 18.0
Saint_Two_Face: 0.0
Saint_Accountant: 0.0
Saint_Nervous: 0.0
Mat_Player_ver_1: 0.0
Mat_Player_ver_2: 0.0
Mat_Player_ver_3: 2578.0
Adam_M_Player: 5691.0
AdamK_ver_1: 253.0
AdamK2_ver_2: 2281.0
Saint_Collector: 564.0
Dabbler_Believer: 3077.0
Dabbler_Once_in_a_While: 3655.0
Dabbler_Two_Face: 6376.0
Dabbler_Accountant: 5256.0
Dabbler_Nervous: 2091.0
Dabbler_Collector: 3297.0
Coin_Flipper_Believer: 8502.0
Coin_Flipper_Once_in_a_While: 8867.0
Coin_Flipper_Two_Face: 9801.0
Coin_Flipper_Accountant: 9396.0
Coin_Flipper_Nervous: 8113.0
Coin_Flipper_Collector: 8646.0
Pathological_Believe

In [15]:
for col, val in results.loc["Janek_Michal_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 6570.0
Saint_Sceptic_25_Naive: 9196.0
Saint_Sceptic_50_Naive: 9287.0
Saint_Sceptic_75_Naive: 9358.0
Saint_Sceptic_Adaptable_Naive: 8442.0
Mat_Player_ver_4: 6052.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8065.0
MyRandom_All_Cards: 9673.0
MyRandom_Real_Cards: 4252.0
Random_Never_Accuse: 8707.0
Random_Always_Accuse: 5607.0
Mimic: 9947.0
Saint_Believer: 7474.0
Saint_Once_in_a_While: 7461.0
Saint_Two_Face: 7937.0
Saint_Accountant: 7899.0
Saint_Nervous: 6091.0
Mat_Player_ver_1: 6035.0
Mat_Player_ver_2: 6026.0
Mat_Player_ver_3: 6120.0
Adam_M_Player: 7988.0
AdamK_ver_1: 5879.0
AdamK2_ver_2: 6571.0
Saint_Collector: 7379.0
Dabbler_Believer: 8834.0
Dabbler_Once_in_a_While: 8592.0
Dabbler_Two_Face: 7058.0
Dabbler_Accountant: 6993.0
Dabbler_Nervous: 6118.0
Dabbler_Collector: 8857.0
Coin_Flipper_Believer: 8871.0
Coin_Flipper_Once_in_a_While: 8436.0
Coin_Flipper_Two_Face: 6472.0
Coin_Flipper_Accountant: 6328.0
Coin_Flipper_Nervous: 6192.0
Coin_Flipper_Collector: 880

In [16]:
for col, val in results.loc["Vadym_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 6130.0
Saint_Sceptic_25_Naive: 4297.0
Saint_Sceptic_50_Naive: 4931.0
Saint_Sceptic_75_Naive: 5706.0
Saint_Sceptic_Adaptable_Naive: 7098.0
Mat_Player_ver_4: 800.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8280.0
MyRandom_All_Cards: 9867.0
MyRandom_Real_Cards: 4728.0
Random_Never_Accuse: 9443.0
Random_Always_Accuse: 903.0
Mimic: 9991.0
Saint_Believer: 7339.0
Saint_Once_in_a_While: 6499.0
Saint_Two_Face: 4586.0
Saint_Accountant: 5018.0
Saint_Nervous: 876.0
Mat_Player_ver_1: 891.0
Mat_Player_ver_2: 884.0
Mat_Player_ver_3: 916.0
Adam_M_Player: 3211.0
AdamK_ver_1: 5748.0
AdamK2_ver_2: 6313.0
Saint_Collector: 7034.0
Dabbler_Believer: 9323.0
Dabbler_Once_in_a_While: 8991.0
Dabbler_Two_Face: 6160.0
Dabbler_Accountant: 4786.0
Dabbler_Nervous: 4797.0
Dabbler_Collector: 9240.0
Coin_Flipper_Believer: 9342.0
Coin_Flipper_Once_in_a_While: 9087.0
Coin_Flipper_Two_Face: 6151.0
Coin_Flipper_Accountant: 4597.0
Coin_Flipper_Nervous: 5337.0
Coin_Flipper_Collector: 9314.0
Pa

In [17]:
for col, val in results.loc["Krzysztof_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9919.0
Saint_Sceptic_25_Naive: 5011.0
Saint_Sceptic_50_Naive: 4943.0
Saint_Sceptic_75_Naive: 5004.0
Saint_Sceptic_Adaptable_Naive: 0.0
Mat_Player_ver_4: 0.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 10000.0
Random_Never_Accuse: 10000.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 0.0
Saint_Once_in_a_While: 0.0
Saint_Two_Face: 0.0
Saint_Accountant: 0.0
Saint_Nervous: 0.0
Mat_Player_ver_1: 0.0
Mat_Player_ver_2: 0.0
Mat_Player_ver_3: 811.0
Adam_M_Player: 5634.0
AdamK_ver_1: 279.0
AdamK2_ver_2: 305.0
Saint_Collector: 130.0
Dabbler_Believer: 2815.0
Dabbler_Once_in_a_While: 2877.0
Dabbler_Two_Face: 3728.0
Dabbler_Accountant: 4572.0
Dabbler_Nervous: 1434.0
Dabbler_Collector: 2665.0
Coin_Flipper_Believer: 8813.0
Coin_Flipper_Once_in_a_While: 9088.0
Coin_Flipper_Two_Face: 9922.0
Coin_Flipper_Accountant: 9578.0
Coin_Flipper_Nervous: 8614.0
Coin_Flipper_Collector: 8909.0
Pathological_Believer

In [18]:
for col, val in results.loc["Adam_M_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9985.0
Saint_Sceptic_25_Naive: 4321.0
Saint_Sceptic_50_Naive: 4276.0
Saint_Sceptic_75_Naive: 4348.0
Saint_Sceptic_Adaptable_Naive: 61.0
Mat_Player_ver_4: 15.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9999.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9998.0
Random_Never_Accuse: 9982.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 77.0
Saint_Once_in_a_While: 61.0
Saint_Two_Face: 11.0
Saint_Accountant: 40.0
Saint_Nervous: 13.0
Mat_Player_ver_1: 11.0
Mat_Player_ver_2: 11.0
Mat_Player_ver_3: 833.0
Adam_M_Player: nan
AdamK_ver_1: 649.0
AdamK2_ver_2: 604.0
Saint_Collector: 90.0
Dabbler_Believer: 5423.0
Dabbler_Once_in_a_While: 6513.0
Dabbler_Two_Face: 9409.0
Dabbler_Accountant: 7697.0
Dabbler_Nervous: 4008.0
Dabbler_Collector: 5610.0
Coin_Flipper_Believer: 9147.0
Coin_Flipper_Once_in_a_While: 9561.0
Coin_Flipper_Two_Face: 9976.0
Coin_Flipper_Accountant: 9852.0
Coin_Flipper_Nervous: 9005.0
Coin_Flipper_Collector: 9183.0
Pathological_Believ

In [19]:
for col, va in results.loc["AdamK_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 32.0
Saint_Sceptic_25_Naive: 32.0
Saint_Sceptic_50_Naive: 32.0
Saint_Sceptic_75_Naive: 32.0
Saint_Sceptic_Adaptable_Naive: 32.0
Mat_Player_ver_4: 32.0
YourName - not implemented Player: 32.0
MyRandom_Default: 32.0
MyRandom_All_Cards: 32.0
MyRandom_Real_Cards: 32.0
Random_Never_Accuse: 32.0
Random_Always_Accuse: 32.0
Mimic: 32.0
Saint_Believer: 32.0
Saint_Once_in_a_While: 32.0
Saint_Two_Face: 32.0
Saint_Accountant: 32.0
Saint_Nervous: 32.0
Mat_Player_ver_1: 32.0
Mat_Player_ver_2: 32.0
Mat_Player_ver_3: 32.0
Adam_M_Player: 32.0
AdamK_ver_1: 32.0
AdamK2_ver_2: 32.0
Saint_Collector: 32.0
Dabbler_Believer: 32.0
Dabbler_Once_in_a_While: 32.0
Dabbler_Two_Face: 32.0
Dabbler_Accountant: 32.0
Dabbler_Nervous: 32.0
Dabbler_Collector: 32.0
Coin_Flipper_Believer: 32.0
Coin_Flipper_Once_in_a_While: 32.0
Coin_Flipper_Two_Face: 32.0
Coin_Flipper_Accountant: 32.0
Coin_Flipper_Nervous: 32.0
Coin_Flipper_Collector: 32.0
Pathological_Believer: 32.0
Pathological_Once_in_a_While: 32.0
Patholog

In [20]:
for col, val in results.loc["AdamK2_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 9264.0
Saint_Sceptic_25_Naive: 9531.0
Saint_Sceptic_50_Naive: 9522.0
Saint_Sceptic_75_Naive: 9635.0
Saint_Sceptic_Adaptable_Naive: 7725.0
Mat_Player_ver_4: 3201.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9568.0
MyRandom_All_Cards: 9958.0
MyRandom_Real_Cards: 8419.0
Random_Never_Accuse: 8647.0
Random_Always_Accuse: 9170.0
Mimic: 9991.0
Saint_Believer: 6044.0
Saint_Once_in_a_While: 6778.0
Saint_Two_Face: 8436.0
Saint_Accountant: 8470.0
Saint_Nervous: 3692.0
Mat_Player_ver_1: 3657.0
Mat_Player_ver_2: 3706.0
Mat_Player_ver_3: 5418.0
Adam_M_Player: 9396.0
AdamK_ver_1: 4457.0
AdamK2_ver_2: nan
Saint_Collector: 5958.0
Dabbler_Believer: 8194.0
Dabbler_Once_in_a_While: 8417.0
Dabbler_Two_Face: 9364.0
Dabbler_Accountant: 8990.0
Dabbler_Nervous: 6827.0
Dabbler_Collector: 8102.0
Coin_Flipper_Believer: 8398.0
Coin_Flipper_Once_in_a_While: 8499.0
Coin_Flipper_Two_Face: 9267.0
Coin_Flipper_Accountant: 9000.0
Coin_Flipper_Nervous: 7321.0
Coin_Flipper_Collector: 8337.0

In [21]:
for col, val in results.loc["B_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 8560.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 9999.0
Saint_Sceptic_Adaptable_Naive: 6483.0
Mat_Player_ver_4: 3422.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9516.0
MyRandom_All_Cards: 9959.0
MyRandom_Real_Cards: 6209.0
Random_Never_Accuse: 3555.0
Random_Always_Accuse: 9917.0
Mimic: 10000.0
Saint_Believer: 4221.0
Saint_Once_in_a_While: 5734.0
Saint_Two_Face: 8843.0
Saint_Accountant: 8143.0
Saint_Nervous: 3490.0
Mat_Player_ver_1: 3527.0
Mat_Player_ver_2: 3525.0
Mat_Player_ver_3: 6400.0
Adam_M_Player: 9955.0
AdamK_ver_1: 4177.0
AdamK2_ver_2: 5206.0
Saint_Collector: 4296.0
Dabbler_Believer: 4184.0
Dabbler_Once_in_a_While: 4923.0
Dabbler_Two_Face: 9125.0
Dabbler_Accountant: 5704.0
Dabbler_Nervous: 3166.0
Dabbler_Collector: 4182.0
Coin_Flipper_Believer: 3866.0
Coin_Flipper_Once_in_a_While: 4612.0
Coin_Flipper_Two_Face: 8556.0
Coin_Flipper_Accountant: 5194.0
Coin_Flipper_Nervous: 3053.0
Coin_Flipper_Collector: 

In [22]:
for col, val in results.loc["B_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 8235.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 6395.0
Mat_Player_ver_4: 3572.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9368.0
MyRandom_All_Cards: 9961.0
MyRandom_Real_Cards: 4959.0
Random_Never_Accuse: 3322.0
Random_Always_Accuse: 9919.0
Mimic: 10000.0
Saint_Believer: 4160.0
Saint_Once_in_a_While: 5708.0
Saint_Two_Face: 8785.0
Saint_Accountant: 8112.0
Saint_Nervous: 3566.0
Mat_Player_ver_1: 3459.0
Mat_Player_ver_2: 3393.0
Mat_Player_ver_3: 6373.0
Adam_M_Player: 9958.0
AdamK_ver_1: 4106.0
AdamK2_ver_2: 5205.0
Saint_Collector: 4206.0
Dabbler_Believer: 3928.0
Dabbler_Once_in_a_While: 4831.0
Dabbler_Two_Face: 8903.0
Dabbler_Accountant: 5449.0
Dabbler_Nervous: 2949.0
Dabbler_Collector: 4102.0
Coin_Flipper_Believer: 3792.0
Coin_Flipper_Once_in_a_While: 4320.0
Coin_Flipper_Two_Face: 8227.0
Coin_Flipper_Accountant: 4939.0
Coin_Flipper_Nervous: 2785.0
Coin_Flipper_Collector:

In [23]:
for col, val in results.loc["B_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 8969.0
Saint_Sceptic_25_Naive: 9286.0
Saint_Sceptic_50_Naive: 9305.0
Saint_Sceptic_75_Naive: 9265.0
Saint_Sceptic_Adaptable_Naive: 9118.0
Mat_Player_ver_4: 4576.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 8979.0
MyRandom_All_Cards: 9878.0
MyRandom_Real_Cards: 7347.0
Random_Never_Accuse: 8733.0
Random_Always_Accuse: 7760.0
Mimic: 9875.0
Saint_Believer: 7522.0
Saint_Once_in_a_While: 7949.0
Saint_Two_Face: 9230.0
Saint_Accountant: 9477.0
Saint_Nervous: 6219.0
Mat_Player_ver_1: 5436.0
Mat_Player_ver_2: 5370.0
Mat_Player_ver_3: 5265.0
Adam_M_Player: 9318.0
AdamK_ver_1: 5220.0
AdamK2_ver_2: 5670.0
Saint_Collector: 7476.0
Dabbler_Believer: 8879.0
Dabbler_Once_in_a_While: 8757.0
Dabbler_Two_Face: 9191.0
Dabbler_Accountant: 9531.0
Dabbler_Nervous: 7329.0
Dabbler_Collector: 8887.0
Coin_Flipper_Believer: 8832.0
Coin_Flipper_Once_in_a_While: 8584.0
Coin_Flipper_Two_Face: 8939.0
Coin_Flipper_Accountant: 9312.0
Coin_Flipper_Nervous: 7224.0
Coin_Flipper_Collector: 883

In [24]:
for col, val in results.loc["Bartek_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8201.0
Mat_Player_ver_4: 4925.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9998.0
Random_Never_Accuse: 9908.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5927.0
Saint_Once_in_a_While: 7706.0
Saint_Two_Face: 9953.0
Saint_Accountant: 9464.0
Saint_Nervous: 5053.0
Mat_Player_ver_1: 5118.0
Mat_Player_ver_2: 4912.0
Mat_Player_ver_3: 7489.0
Adam_M_Player: 9981.0
AdamK_ver_1: 6452.0
AdamK2_ver_2: 6712.0
Saint_Collector: 5985.0
Dabbler_Believer: 8693.0
Dabbler_Once_in_a_While: 9463.0
Dabbler_Two_Face: 9997.0
Dabbler_Accountant: 9871.0
Dabbler_Nervous: 8384.0
Dabbler_Collector: 8734.0
Coin_Flipper_Believer: 9481.0
Coin_Flipper_Once_in_a_While: 9805.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9982.0
Coin_Flipper_Nervous: 9444.0
Coin_Flipper_Collect

In [25]:
for col, val in results.loc["Bartek_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 10000.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8195.0
Mat_Player_ver_4: 5061.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9905.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 6008.0
Saint_Once_in_a_While: 7726.0
Saint_Two_Face: 9963.0
Saint_Accountant: 9452.0
Saint_Nervous: 5068.0
Mat_Player_ver_1: 4945.0
Mat_Player_ver_2: 4992.0
Mat_Player_ver_3: 7418.0
Adam_M_Player: 9988.0
AdamK_ver_1: 6484.0
AdamK2_ver_2: 6642.0
Saint_Collector: 6086.0
Dabbler_Believer: 8724.0
Dabbler_Once_in_a_While: 9464.0
Dabbler_Two_Face: 9997.0
Dabbler_Accountant: 9866.0
Dabbler_Nervous: 8440.0
Dabbler_Collector: 8763.0
Coin_Flipper_Believer: 9480.0
Coin_Flipper_Once_in_a_While: 9798.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9981.0
Coin_Flipper_Nervous: 9477.0
Coin_Flipper_Collec

In [26]:
for col, val in results.loc["Bartek_ver_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 9349.0
Saint_Sceptic_25_Naive: 9275.0
Saint_Sceptic_50_Naive: 9276.0
Saint_Sceptic_75_Naive: 9251.0
Saint_Sceptic_Adaptable_Naive: 9705.0
Mat_Player_ver_4: 4647.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9256.0
MyRandom_All_Cards: 9955.0
MyRandom_Real_Cards: 7963.0
Random_Never_Accuse: 9733.0
Random_Always_Accuse: 7987.0
Mimic: 9923.0
Saint_Believer: 7812.0
Saint_Once_in_a_While: 8222.0
Saint_Two_Face: 9417.0
Saint_Accountant: 9637.0
Saint_Nervous: 6379.0
Mat_Player_ver_1: 5457.0
Mat_Player_ver_2: 5449.0
Mat_Player_ver_3: 5382.0
Adam_M_Player: 9410.0
AdamK_ver_1: 5288.0
AdamK2_ver_2: 5821.0
Saint_Collector: 7871.0
Dabbler_Believer: 9745.0
Dabbler_Once_in_a_While: 9513.0
Dabbler_Two_Face: 9488.0
Dabbler_Accountant: 9777.0
Dabbler_Nervous: 7934.0
Dabbler_Collector: 9750.0
Coin_Flipper_Believer: 9755.0
Coin_Flipper_Once_in_a_While: 9450.0
Coin_Flipper_Two_Face: 9324.0
Coin_Flipper_Accountant: 9703.0
Coin_Flipper_Nervous: 8035.0
Coin_Flipper_Collector: 970

In [27]:
for col, val in results.loc["Bartek_ver_4"].items():
    print(f'{col}: {val}')

RandomPlayer: 9278.0
Saint_Sceptic_25_Naive: 9322.0
Saint_Sceptic_50_Naive: 9240.0
Saint_Sceptic_75_Naive: 9258.0
Saint_Sceptic_Adaptable_Naive: 9708.0
Mat_Player_ver_4: 4531.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9145.0
MyRandom_All_Cards: 9953.0
MyRandom_Real_Cards: 7534.0
Random_Never_Accuse: 9708.0
Random_Always_Accuse: 8026.0
Mimic: 9924.0
Saint_Believer: 7836.0
Saint_Once_in_a_While: 8246.0
Saint_Two_Face: 9365.0
Saint_Accountant: 9612.0
Saint_Nervous: 6390.0
Mat_Player_ver_1: 5462.0
Mat_Player_ver_2: 5590.0
Mat_Player_ver_3: 5491.0
Adam_M_Player: 9308.0
AdamK_ver_1: 5275.0
AdamK2_ver_2: 5880.0
Saint_Collector: 7837.0
Dabbler_Believer: 9759.0
Dabbler_Once_in_a_While: 9504.0
Dabbler_Two_Face: 9466.0
Dabbler_Accountant: 9797.0
Dabbler_Nervous: 7914.0
Dabbler_Collector: 9735.0
Coin_Flipper_Believer: 9757.0
Coin_Flipper_Once_in_a_While: 9418.0
Coin_Flipper_Two_Face: 9267.0
Coin_Flipper_Accountant: 9696.0
Coin_Flipper_Nervous: 8043.0
Coin_Flipper_Collector: 969

In [28]:
for col, val in results.loc["Mat_Player_ver_1"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8273.0
Mat_Player_ver_4: 4964.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9999.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9913.0
Random_Always_Accuse: 10000.0
Mimic: 9650.0
Saint_Believer: 5859.0
Saint_Once_in_a_While: 7729.0
Saint_Two_Face: 9962.0
Saint_Accountant: 9406.0
Saint_Nervous: 4976.0
Mat_Player_ver_1: nan
Mat_Player_ver_2: 5057.0
Mat_Player_ver_3: 7550.0
Adam_M_Player: 9989.0
AdamK_ver_1: 6445.0
AdamK2_ver_2: 6343.0
Saint_Collector: 5966.0
Dabbler_Believer: 8517.0
Dabbler_Once_in_a_While: 9371.0
Dabbler_Two_Face: 9996.0
Dabbler_Accountant: 9818.0
Dabbler_Nervous: 8198.0
Dabbler_Collector: 8557.0
Coin_Flipper_Believer: 9389.0
Coin_Flipper_Once_in_a_While: 9769.0
Coin_Flipper_Two_Face: 9998.0
Coin_Flipper_Accountant: 9956.0
Coin_Flipper_Nervous: 9403.0
Coin_Flipper_Collector: 9

In [29]:
for col, val in results.loc["Mat_Player_ver_2"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8207.0
Mat_Player_ver_4: 4932.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9998.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 10000.0
Random_Never_Accuse: 9902.0
Random_Always_Accuse: 10000.0
Mimic: 9630.0
Saint_Believer: 5976.0
Saint_Once_in_a_While: 7652.0
Saint_Two_Face: 9957.0
Saint_Accountant: 9415.0
Saint_Nervous: 4952.0
Mat_Player_ver_1: 4943.0
Mat_Player_ver_2: nan
Mat_Player_ver_3: 7451.0
Adam_M_Player: 9989.0
AdamK_ver_1: 6449.0
AdamK2_ver_2: 6294.0
Saint_Collector: 5966.0
Dabbler_Believer: 8435.0
Dabbler_Once_in_a_While: 9388.0
Dabbler_Two_Face: 9994.0
Dabbler_Accountant: 9847.0
Dabbler_Nervous: 8220.0
Dabbler_Collector: 8502.0
Coin_Flipper_Believer: 9432.0
Coin_Flipper_Once_in_a_While: 9755.0
Coin_Flipper_Two_Face: 9998.0
Coin_Flipper_Accountant: 9973.0
Coin_Flipper_Nervous: 9427.0
Coin_Flipper_Collector: 

In [30]:
for col, val in results.loc["Mat_Player_ver_3"].items():
    print(f'{col}: {val}')

RandomPlayer: 9999.0
Saint_Sceptic_25_Naive: 9183.0
Saint_Sceptic_50_Naive: 9180.0
Saint_Sceptic_75_Naive: 9163.0
Saint_Sceptic_Adaptable_Naive: 5844.0
Mat_Player_ver_4: 2582.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9998.0
Random_Never_Accuse: 9915.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 5073.0
Saint_Once_in_a_While: 5680.0
Saint_Two_Face: 9266.0
Saint_Accountant: 6549.0
Saint_Nervous: 2494.0
Mat_Player_ver_1: 2450.0
Mat_Player_ver_2: 2549.0
Mat_Player_ver_3: nan
Adam_M_Player: 9167.0
AdamK_ver_1: 4624.0
AdamK2_ver_2: 4582.0
Saint_Collector: 5313.0
Dabbler_Believer: 8028.0
Dabbler_Once_in_a_While: 9077.0
Dabbler_Two_Face: 9996.0
Dabbler_Accountant: 9703.0
Dabbler_Nervous: 7423.0
Dabbler_Collector: 8095.0
Coin_Flipper_Believer: 9334.0
Coin_Flipper_Once_in_a_While: 9744.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9953.0
Coin_Flipper_Nervous: 9305.0
Coin_Flipper_Collector: 94

In [31]:
for col, val in results.loc["Mat_Player_ver_4"].items():
    print(f'{col}: {val}')

RandomPlayer: 9998.0
Saint_Sceptic_25_Naive: 10000.0
Saint_Sceptic_50_Naive: 10000.0
Saint_Sceptic_75_Naive: 10000.0
Saint_Sceptic_Adaptable_Naive: 8256.0
Mat_Player_ver_4: nan
YourName - not implemented Player: 10000.0
MyRandom_Default: 10000.0
MyRandom_All_Cards: 10000.0
MyRandom_Real_Cards: 9999.0
Random_Never_Accuse: 9910.0
Random_Always_Accuse: 10000.0
Mimic: 10000.0
Saint_Believer: 6010.0
Saint_Once_in_a_While: 7775.0
Saint_Two_Face: 9963.0
Saint_Accountant: 9442.0
Saint_Nervous: 5011.0
Mat_Player_ver_1: 5036.0
Mat_Player_ver_2: 5068.0
Mat_Player_ver_3: 7418.0
Adam_M_Player: 9985.0
AdamK_ver_1: 6544.0
AdamK2_ver_2: 6799.0
Saint_Collector: 6054.0
Dabbler_Believer: 8620.0
Dabbler_Once_in_a_While: 9468.0
Dabbler_Two_Face: 9995.0
Dabbler_Accountant: 9870.0
Dabbler_Nervous: 8433.0
Dabbler_Collector: 8735.0
Coin_Flipper_Believer: 9530.0
Coin_Flipper_Once_in_a_While: 9805.0
Coin_Flipper_Two_Face: 9999.0
Coin_Flipper_Accountant: 9973.0
Coin_Flipper_Nervous: 9490.0
Coin_Flipper_Collector:

In [32]:
for col, val in results.loc["Samuel_Player"].items():
    print(f'{col}: {val}')

RandomPlayer: 9785.0
Saint_Sceptic_25_Naive: 9588.0
Saint_Sceptic_50_Naive: 9567.0
Saint_Sceptic_75_Naive: 9609.0
Saint_Sceptic_Adaptable_Naive: 7967.0
Mat_Player_ver_4: 5949.0
YourName - not implemented Player: 10000.0
MyRandom_Default: 9963.0
MyRandom_All_Cards: 9988.0
MyRandom_Real_Cards: 9725.0
Random_Never_Accuse: 8531.0
Random_Always_Accuse: 9994.0
Mimic: 10000.0
Saint_Believer: 6964.0
Saint_Once_in_a_While: 7021.0
Saint_Two_Face: 8912.0
Saint_Accountant: 8634.0
Saint_Nervous: 5969.0
Mat_Player_ver_1: 5955.0
Mat_Player_ver_2: 5965.0
Mat_Player_ver_3: 6192.0
Adam_M_Player: 9613.0
AdamK_ver_1: 5814.0
AdamK2_ver_2: 6734.0
Saint_Collector: 6949.0
Dabbler_Believer: 7940.0
Dabbler_Once_in_a_While: 8280.0
Dabbler_Two_Face: 9618.0
Dabbler_Accountant: 9205.0
Dabbler_Nervous: 6868.0
Dabbler_Collector: 7918.0
Coin_Flipper_Believer: 8559.0
Coin_Flipper_Once_in_a_While: 8913.0
Coin_Flipper_Two_Face: 9808.0
Coin_Flipper_Accountant: 9781.0
Coin_Flipper_Nervous: 7969.0
Coin_Flipper_Collector: 85

### Results - Only Student's Bots

In [35]:
agent_strategies = [
            ["Mat_Player_ver_4", "ENSAMBLE", ensamble_player],
           ["Mat_Player_ver_1", "SAINT", Saint_Nervous_ver_2],
           ["Mat_Player_ver_2", "SAINT", Saint_Nervous_ver_3],
           ["Mat_Player_ver_3", "SAINT", Saint_Nervous_Memory],
           ["Adam_M_Player", "CUSTOM", Adam_player],
           ["AdamK_ver_1", "CUSTOM", AdamK],
           ["AdamK2_ver_2", "CUSTOM", AdamK2],
           ["Bartek_ver_1", "B_PLAYER", Saint_Nervous_ver_B],
           ["Bartek_ver_2", "B_PLAYER", Saint_Nervous_ver_B2],
           ["Bartek_ver_3", "B_PLAYER", Saint_Nervous_ver_B3],
           ["Bartek_ver_4", "B_PLAYER", Saint_Nervous_ver_B4],
           ["Janek_Michal_ver_1", "MICHAL_JANEK", KaminskiIndrzejczak_ver_1],
           ["Janek_Michal_ver_2", "MICHAL_JANEK", KaminskiIndrzejczak_ver_2],
           ["Vadym_Player", "VADYM", Vadym_Player],
           ["Krzysztof_Player", "KRZYSZTOF", Krzysztof_Player],
           ["Samuel_Player", "SAMUEL", Amalgamat]
           ]

results = compare_all(agent_strategies)
results

,Mat_Player_ver_4,Mat_Player_ver_1,Mat_Player_ver_2,Mat_Player_ver_3,Adam_M_Player,AdamK_ver_1,AdamK2_ver_2,Bartek_ver_1,Bartek_ver_2,Bartek_ver_3,Bartek_ver_4,Janek_Michal_ver_1,Janek_Michal_ver_2,Vadym_Player,Krzysztof_Player,Samuel_Player,|| Total Wins ||,|| Dominated Players ||
Samuel_Player,5881.0,5981.0,5935.0,6170.0,9577.0,5921.0,6696.0,5979.0,5869.0,5850.0,5701.0,9744.0,4687.0,6169.0,9624.0,NaN,99784.0,14
Mat_Player_ver_4,NaN,5008.0,5003.0,7456.0,9990.0,6487.0,6724.0,4989.0,5012.0,5443.0,5381.0,10000.0,3953.0,9231.0,10000.0,4119.0,98796.0,12
Bartek_ver_2,4988.0,5025.0,4913.0,7542.0,9993.0,6557.0,6808.0,4916.0,NaN,5316.0,5414.0,10000.0,4018.0,9169.0,10000.0,4131.0,98790.0,10
Bartek_ver_1,5011.0,5101.0,4948.0,7496.0,9990.0,6449.0,6732.0,NaN,5084.0,5302.0,5416.0,10000.0,3999.0,9212.0,10000.0,4021.0,98761.0,12
Janek_Michal_ver_2,6047.0,5940.0,5950.0,6082.0,8012.0,5944.0,6673.0,6001.0,5982.0,6168.0,6001.0,9472.0,NaN,5211.0,9197.0,5313.0,97993.0,15
Mat_Player_ver_2,4997.0,4952.0,NaN,7453.0,9996.0,6390.0,6265.0,5052.0,5087.0,4517.0,4513.0,10000.0,4050.0,9181.0,10000.0,4065.0,96518.0,9
Mat_Player_ver_1,4992.0,NaN,5048.0,7455.0,9989.0,6349.0,6245.0,4899.0,4975.0,4461.0,4594.0,10000.0,4060.0,9172.0,10000.0,4019.0,96258.0,8
Bartek_ver_3,4557.0,5539.0,5483.0,5436.0,9415.0,5324.0,5785.0,4698.0,4684.0,NaN,5002.0,9656.0,3832.0,3929.0,9227.0,4150.0,86717.0,9
Bartek_ver_4,4619.0,5406.0,5487.0,5382.0,9363.0,5246.0,5774.0,4584.0,4586.0,4998.0,NaN,9551.0,3999.0,3796.0,9246.0,4299.0,86336.0,8
AdamK_ver_1,3513.0,3651.0,3610.0,5338.0,9357.0,NaN,5466.0,3551.0,3443.0,4676.0,4754.0,9722.0,4056.0,4281.0,9722.0,4079.0,79219.0,5
